<a href="https://colab.research.google.com/github/Bayron077/Bayron077/blob/main/ModeloIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers datasets accelerate -q

In [5]:
from datasets import load_dataset

# Usamos "wikimedia/wikipedia" en español, más estable y confiable
dataset = load_dataset("wikimedia/wikipedia", "20231101.es", split="train", streaming=True)

# Tomamos solo 3000 ejemplos para que no tarde mucho
dataset = dataset.take(3000)

# Convertimos a lista
data_list = list(dataset)
print(f"Total de ejemplos: {len(data_list)}")
print("Ejemplo de texto:")
print(data_list[0]['text'][:300])

README.md: 0.00B [00:00, ?B/s]

Total de ejemplos: 3000
Ejemplo de texto:
Andorra, oficialmente Principado de Andorra () es un micro-Estado soberano sin litoral ubicado en el suroeste de Europa, entre España y Francia, en el límite de la península ibérica. Se constituye en Estado independiente, de derecho, democrático y social, cuya forma de gobierno es el coprincipado pa


In [7]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

# Usamos GPT-2 entrenado en español
model_name = "datificate/gpt2-small-spanish"

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

print("✅ Modelo y tokenizador cargados correctamente")
print(f"Parámetros del modelo: {model.num_parameters():,}")

tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/510M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: datificate/gpt2-small-spanish
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Modelo y tokenizador cargados correctamente
Parámetros del modelo: 163,037,184


In [8]:
# Guardamos los textos en un archivo .txt
with open("dataset_es.txt", "w", encoding="utf-8") as f:
    for item in data_list:
        texto = item['text'].strip()
        if len(texto) > 100:
            f.write(texto + "\n\n")

print("✅ Archivo de texto guardado")

# Creamos una clase Dataset personalizada (reemplazo moderno de TextDataset)
class TextoDataset(Dataset):
    def __init__(self, tokenizer, archivo, block_size=128):
        with open(archivo, "r", encoding="utf-8") as f:
            texto = f.read()

        tokenizer.pad_token = tokenizer.eos_token
        tokens = tokenizer(texto, return_tensors="pt", truncation=False)["input_ids"][0]

        # Dividimos en bloques del mismo tamaño
        self.ejemplos = []
        for i in range(0, len(tokens) - block_size, block_size):
            self.ejemplos.append(tokens[i : i + block_size])

    def __len__(self):
        return len(self.ejemplos)

    def __getitem__(self, idx):
        return {"input_ids": self.ejemplos[idx], "labels": self.ejemplos[idx]}

train_dataset = TextoDataset(tokenizer, "dataset_es.txt", block_size=128)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

print(f"✅ Dataset tokenizado: {len(train_dataset)} bloques de texto")

✅ Archivo de texto guardado
✅ Dataset tokenizado: 104376 bloques de texto


In [10]:
training_args = TrainingArguments(
    output_dir="./gpt2-spanish-finetuned",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

print("🚀 Iniciando entrenamiento...")
trainer.train()
print("✅ ¡Entrenamiento completado!")

🚀 Iniciando entrenamiento...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,3.825804
200,3.754825
300,3.748120
400,3.772382
500,3.745195
600,3.731444
700,3.728895
800,3.709737
900,3.733004
1000,3.691482


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ ¡Entrenamiento completado!


In [12]:
# Guardamos el modelo y tokenizador manualmente
trainer.save_model("./gpt2-spanish-finetuned")
tokenizer.save_pretrained("./gpt2-spanish-finetuned")
print("✅ Modelo guardado correctamente")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Modelo guardado correctamente


In [16]:
from transformers import pipeline

# Cargamos el modelo que acabamos de entrenar
generador = pipeline("text-generation", model="./gpt2-spanish-finetuned")

# Generamos texto a partir de un inicio
resultado = generador(
    "El gobierno anunció hoy que",
    max_length=100,
    num_return_sequences=1,
    temperature=0.3,
    do_sample=True
)

print("📝 Texto generado:")
print(resultado[0]['generated_text'])

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


📝 Texto generado:
El gobierno anunció hoy que el gobierno de la República de Chile se encuentra en crisis y que el presidente de la República, Sebastián Piñera, se ha convertido en el nuevo presidente de la República.

El 2 de agosto de 2014, el presidente Sebastián Piñera anunció que el gobierno de la República de Chile se encuentra en crisis y que el presidente de la República, Sebastián Piñera, se ha convertido en el nuevo presidente de la República de Chile.

El 2 de agosto de 2014, el presidente Sebastián Piñera anunció que el gobierno de la República de Chile se encuentra en crisis y que el presidente de la República, Sebastián Piñera, se ha convertido en el nuevo presidente de la República de Chile.

El 3 de agosto de 2014, el presidente Sebastián Piñera anunció que el gobierno de la República de Chile se encuentra en crisis y que el presidente de la República, Sebastián Piñera, se ha convertido en el nuevo presidente de la República de Chile.

El 3 de agosto de 2015, el preside

In [17]:
frases = [
    "La inteligencia artificial en el futuro",
    "Colombia es un país conocido por",
    "Los científicos descubrieron que",
    "En el año 2050 la humanidad",
]

for frase in frases:
    resultado = generador(
        frase,
        max_length=80,
        do_sample=True,
        temperature=0.9,
    )
    print(f"\n📝 Inicio: '{frase}'")
    print(resultado[0]['generated_text'])
    print("-" * 60)

Passing `generation_config` together with generation-related arguments=({'max_length', 'do_sample', 'temperature'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📝 Inicio: 'La inteligencia artificial en el futuro'
La inteligencia artificial en el futuro

La inteligencia artificial en el futuro, se define como un sistema dinámico que controla el futuro. Se denomina inteligencia artificial a la rama de la ciencia que estudia la inteligencia artificial y cómo se puede manipularla para ayudar a los administradores o trabajadores de una empresa o empresa a desarrollar sus productos.

A partir de esta ciencia, todos los programas de inteligencia artificial deben ser realizados por personas que no pueden acceder a un sistema o sistema de procesamiento completo, pero el objetivo principal de una entidad es proporcionar información que contribuya a la mejora de su máquina.

La parte principal de la inteligencia artificial se denomina inteligencia empresarial. En sentido estricto, la inteligencia empresarial es un conjunto de métodos de comunicación en línea que se pueden transmitir con el sistema nervioso, para transferir la información existente a los

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📝 Inicio: 'Colombia es un país conocido por'
Colombia es un país conocido por su producción de vino moscatel, espárragos de uva de mesa, espárragos de uva, vinos de mesa, melón y atún. También se producen algunos vinos que, en época de crisis económica, en la actualidad se consideran productos de exportación de Europa.

Según una encuesta realizada por la Asociación de Productores de vino del Gobierno de España al año 2005, la elaboración de vino en el País Vasco (España) —con una producción por sobre el 50 % de la producción mundial— sigue siendo la producción mundial de vino, aunque en 2007 la producción de vino en Castilla y León ya estaba en primer lugar. Por otra parte, en 2008 el número de productores de vino en Castilla y León seguía siendo el de la Comunidad de Madrid, hasta que fue sustituido por el de León.

Los principales vinos producidos en Castilla y León son la uva de mesa, la uva de mesa, el vino de mesa dulce de mesa (variedad de Burdeos, de la Denia, de San Millán, d

Both `max_new_tokens` (=256) and `max_length`(=80) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



📝 Inicio: 'Los científicos descubrieron que'
Los científicos descubrieron que los rayos X y rayos gamma pueden ser detectados a simple vista mediante el telescopio espacial Hubble. Los rayos X de emisión de rayos gamma pueden detectarse en todos los materiales a través de la observación de objetos a través de la atmósfera terrestre, como agua, polvo y rocas.

En el año 1996, el estudio de materiales atmosféricos (ETS) se introdujo en el campo de la cosmología. Los átomos, moléculas, moléculas y sólidos más pesados o pesados son clasificados según su número atómico atómico. Los átomos del núcleo y de los átomos moleculares más pesados representan los átomos más pesados de la tabla periódica; por lo tanto, en la física clásica, los átomos pesados tienen la misma masa estructural.

Los rayos X pueden detectarse a gran distancia por la observación de objetos pequeños, como la atmósfera terrestre. Un objeto de cierta magnitud, como un asteroide o un volcán, puede ser detectado a simple vis